# Split text and tag

Smoker and Family_hist can have no tag

In [175]:
#according to flair corpus "UD_ENGLISH", train:test:dev = 6:1:1
from os import listdir
from sklearn.model_selection import train_test_split
file_path_1 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
file_path_2 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set2/"
file_path_3 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/testing-RiskFactors-Complete/"
names_1 = [f for f in listdir(file_path_1) if f.endswith('.xml')]
names_2 = [f for f in listdir(file_path_2) if f.endswith('.xml')]
names_3 = [f for f in listdir(file_path_3) if f.endswith('.xml')] 


In [176]:


import pandas as pd
import numpy as np
#train_1_df = pd.DataFrame(np.zeros((len(train_n_1), 2), dtype=object), columns=['text', 'annotation'])
#dev_1_df = pd.DataFrame(np.zeros((len(dev_n_1), 2), dtype=object), columns=['text', 'annotation'])
all1_df = pd.DataFrame(np.zeros((len(names_1), 2), dtype=object), columns=['text', 'annotation'])
all2_df = pd.DataFrame(np.zeros((len(names_2), 2), dtype=object), columns=['text', 'annotation'])
all3_df = pd.DataFrame(np.zeros((len(names_3), 2), dtype=object), columns=['text', 'annotation'])


In [178]:
import re
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET


def to_df(names,df, file_path,PHI_status = True):
    #get text
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()

        ## Get the text
        nt = re.sub('\n',' ',root[0].text)
        nt = re.sub('\t',' ',nt) 
        nt = re.sub('"',"'",nt)
        ## sample 214 has a weird character
        nt = re.sub('>','&gt;',nt) 
        nt = re.sub('<','&lt;',nt)
        ## new wired character
        nt = re.sub('Â',' ',nt)
        nt = re.sub('â',' ',nt)
        nt = re.sub('€',' ',nt)
        nt = re.sub('™',' ',nt)
        df['text'][n] = nt
        n+=1
    
    #get annotations
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()
        ## Get the labels

        tag_list = []
        # get PHI labels if there are any
        if PHI_status == True:
            PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
            for k in range(PHI,len(root[1])):
                tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
        
        # get the rest of labels
        for k in range(len(root[1])):
            for m in range(len(root[1][k])):
                if root[1][k][m].attrib.keys().__contains__('text') == False:
                    continue
                tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
        
        df['annotation'][n] = tag_list
        n+=1
    return df

#train_1 = to_df(train_n_1,train_1_df, file_path_1, PHI_status = False)
#dev_1 = to_df(dev_n_1,dev_1_df, file_path_1, PHI_status = False)
#test_1 = to_df(test_n_1,test_df_1, file_path_1)
#train_2 = to_df(train_n_2,train_df_2, file_path_2, PHI_status = False)
#dev_2 = to_df(dev_n_2,dev_df_2, file_path_2, PHI_status = False)
#test_2 = to_df(test_n_2,test_df_2, file_path_2, PHI_status = False)
#all_1 = to_df(names_1,all1_df, file_path_1, PHI_status = False)
#all_2 = to_df(names_2,all2_df, file_path_2, PHI_status = False)
all_3 = to_df(names_3,all3_df, file_path_3, PHI_status = False)

In [144]:
train = pd.concat([all_1, all_2], ignore_index=True)
test = all_3

# Functions to convert dataframe into input format
520 files takes 3 min\
Please use create_labs function instead of mark_sentences\
source: https://medium.com/thecyphy/training-custom-ner-model-using-flair-df1f9ea9c762


In [182]:
from tqdm import tqdm
from difflib import SequenceMatcher
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string


def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

## replace !!mark_sentence!! to better label the text with more than one word
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def to_txt(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(df.index[df['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)

            #for i in d.keys():
            #    f.writelines(i + ' ' + d[i] +'\n')
            #f.writelines('\n')
            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
def to_csv(df, filepath):
    for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(df.index[df['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)
            d.to_csv(filepath, index = False, header = False, mode = 'a')
            
def main(input,save_path):
    ## An example dataframe.
    #data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
    #              ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
    #              ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
    #              ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])
    data = input
    ## path to save the txt file.
    ## creating the file.
    to_txt(data, save_path)
    
#if __name__ == '__main__':
path = 'C:/Users/Leste/OneDrive - Johns Hopkins/Documents/GitHub/nlpsumm/BERT/data_processed/'

#main(train,path+'train.txt')
main(all_3,path+'test.txt')

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]


# Test which tag is wrong

In [179]:
print(names_3[262])
match_list = []
text = all_3['text'][262]
annotation = all_3['annotation'][262]
for i in annotation:
    a, text_ = matcher(text, i[0])
    print(a)
    match_list.append((a[0][0], a[0][1], i[1]))

231-01.xml
[(974, 990)]
[(974, 990)]
[(974, 990)]
[(2605, 2615)]
[(974, 990)]
[(974, 990)]
[(974, 990)]
[(2605, 2615)]
[(974, 990)]
[(974, 990)]
[(974, 990)]
[(2605, 2615)]
[(756, 759)]
[(756, 759)]
[(756, 759)]
[(756, 759)]
[(1094, 1100)]
[(1094, 1100)]
[(756, 759)]
[(756, 759)]
[(756, 759)]
[(756, 759)]
[(1066, 1073)]
[(1066, 1073)]
[(1066, 1073)]
[(1124, 1136)]
[(1124, 1131)]
[(1066, 1073)]
[(1066, 1073)]
[(1066, 1073)]
[(1094, 1100)]
[(1094, 1100)]
[(1037, 1043)]
[(1037, 1043)]
[(1037, 1043)]
[(1051, 1057)]
[(1051, 1057)]
[(756, 759)]
[(756, 759)]
[(756, 759)]
[(756, 759)]
[(1124, 1136)]
[(1124, 1131)]
[(1066, 1073)]
[(1066, 1073)]
[(1066, 1073)]
[(1094, 1100)]
[(1094, 1100)]
[(1124, 1136)]
[(1124, 1131)]
[(1037, 1043)]
[(1037, 1043)]
[(1037, 1043)]
[(1037, 1043)]
[(1037, 1043)]
[(1037, 1043)]
[(1051, 1057)]
[(1051, 1057)]
[(1051, 1057)]
[(1051, 1057)]
[(222, 225)]
[(222, 225)]
[(222, 225)]
[(222, 225)]
[(222, 225)]
[(1615, 1626)]
[(1619, 1626)]
[(1614, 1626)]
[(222, 225)]
[(222, 2

## Try training NER with BERT

In [139]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# 1. get the corpus (create my corpus)
#file location
data_folder = 'C:/Users/Leste/OneDrive - Johns Hopkins/Documents/GitHub/nlpsumm/BERT/data_processed'

# column format indicating which columns hold the text and label(s)
columns = {0: 'text', 1: 'ner'}

corpus: Corpus = ColumnCorpus(data_folder,columns,
                              train_file = 'all2.txt')


2023-02-04 17:16:37,617 Reading data from C:\Users\Leste\OneDrive - Johns Hopkins\Documents\GitHub\nlpsumm\BERT\data_processed
2023-02-04 17:16:37,619 Train: C:\Users\Leste\OneDrive - Johns Hopkins\Documents\GitHub\nlpsumm\BERT\data_processed\all2.txt
2023-02-04 17:16:37,621 Dev: None
2023-02-04 17:16:37,623 Test: None


In [134]:

# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)


# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True)

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False
                        )
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)


2023-02-04 17:11:10,224 Computing label dictionary. Progress:


1it [00:00, ?it/s]

2023-02-04 17:11:10,233 Dictionary created for label 'ner' with 3 values: SMOKER (seen 1 times), MEDICATION (seen 1 times)


2023-02-04 17:11:17,961 SequenceTagger predicts: Dictionary with 3 tags: <unk>, SMOKER, MEDICATION
